<a href="https://colab.research.google.com/github/jotaanonimo/ChatterBot/blob/master/Dra_Sheyla_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ----------------------------------------------------------------------
#  Dra Sheyla.py
#
#  a cheezy little Eliza knock-off by Joe Strout
#  with some updates by Jeff Epler
#  hacked into a module and updated by Jez Higgins
#
#  Available under MIT licence from https://github.com/jezhiggins/eliza.py
#  This version further tweaked by Zac Hatfield-Dodds.
# ----------------------------------------------------------------------

import json
import re
import random
import string
import time


PATTERNS = [
    [
        "Eu preciso de (.*)",
        [
            "Por que você precisa de %1?",
            "Isso realmente ajudaria você a obter %1?",
            "Tem certeza de que precisa de %1?",
        ],
    ],
    [
        "Por que você não ([^\\?]*)\\??",
        [
            "Você realmente acha que eu não %1?",
            "Talvez eventualmente eu vá %1.",
            "Você realmente quer que eu %1?",
        ],
    ],
    [
        "Por que não posso ([^\\?]*)\\??",
        [
            "Você acha que deveria ser capaz de %1?",
            "Se você pudesse %1, o que você faria?",
            "Eu não sei -- por que você não pode %1?",
            "Você realmente tentou?",
        ],
    ],
    [
        "Não posso (.*)",
        [
            "Como você sabe que não pode %1?",
            "Talvez você pudesse %1 se tentasse.",
            "O que seria necessário para você %1?",
        ],
    ],
    [
"Eu sou (.*)",
        [
            "Você veio até mim porque você é %1?",
            "Há quanto tempo você está %1?",
            "Como você se sente sendo %1?",
        ],
    ],
    [
        "Eu estou (.*)",
        [
            "Como ser %1 faz você se sentir?",
            "Você gosta de ser %1?",
            "Por que você me diz que é %1?",
            "Por que você acha que é %1?",
        ],
    ],
    [
        "Você é ([^\\?]*)\\??",
        [
            "Por que importa se eu sou %1?",
            "Você preferiria se eu não fosse %1?",
            "Talvez você acredite que eu sou %1.",
            "Eu posso ser %1 -- o que você acha?",
        ],
    ],
    [
        "O que (.*)",
        [
            "Por que você pergunta?",
            "Como uma resposta para isso ajudaria você?",
            "O que você acha?",
        ],
    ],
    [
        "Quão (.*)",
        [
            "Como você acha?",
            "Talvez você possa responder sua própria pergunta.",
            "O que você está realmente perguntando?",
        ],
    ],
    [
        "Porque (.*)",
        [
            "Esse é o verdadeiro motivo?",
            "Que outras razões vêm à mente?",
            "Esse motivo se aplica a mais alguma coisa?",
            "Se %1, o que mais deve ser verdade?",
        ],
    ],
    [
        "(.*) desculpe (.*)",
        [
            "Há muitas vezes em que nenhum pedido de desculpas é necessário.",
            "Que sentimentos você tem quando pede desculpas?",
        ],
    ],
    [
        "Olá(.*)",
        [
            "Olá... que bom que você veio hoje.",
            "Olá... como você está hoje?",
            "Olá, como você está se sentindo hoje?",
        ],
    ],
    [
        "Eu penso (.*)",
        ["Você duvida de %1?", "Você realmente acha?", "Mas você não tem certeza de %1?"],
    ],
    [
        "(.*) amigo (.*)",
        [
            "Conte-me mais sobre seus amigos.",
            "Quando você pensa em um amigo, o que vem à mente?",
            "Por que você não me conta sobre um amigo de infância?",
        ],
    ],
    ["Sim", ["Você parece ter certeza.", "OK, mas você pode elaborar um pouco?"]],
    [
        "(.*) computador(.*)",
        [
            "Você está realmente falando de mim?",
            "Parece estranho falar com um computador?",
            "Como os computadores fazem você se sentir?",
            "Você se sente ameaçado por computadores?",
        ],
    ],
    [
        "É isso (.*)",
        [
            "Você acha que é %1?",
            "Talvez seja %1 -- o que você acha?",
            "Se fosse %1, o que você faria?",
            "Pode ser que %1.",
        ],
    ],
    [
        "Isso é (.*)",
        [
            "Você parece muito certo.",
            "Se eu lhe disser que provavelmente não é %1, o que você sentiria?",
        ],
    ],
    [
        "Você pode ([^\\?]*)\\??",
        [
            "O que faz você pensar que eu não posso %1?",
            "Se eu pudesse %1, então o quê?",
            "Por que você pergunta se eu posso %1?",
        ],
    ],
    [
        "Posso ([^\\?]*)\\??",
        [
            "Talvez você não queira %1.",
            "Você quer ser capaz de %1?",
            "Se você pudesse %1, você faria?",
        ],
    ],
    [
        "Tu es (.*)",
        [
            "Por que você acha que eu sou %1?",
            "Você gosta de pensar que eu sou %1?",
            "Talvez você queira que eu seja %1.",
            "Talvez você esteja realmente falando de si mesmo?",
        ],
    ],
    [
        "Você é (.*)",
        [
            "Por que você diz que eu sou %1?",
            "Por que você acha que eu sou %1?",
            "Estamos falando de você ou de mim?",
        ],
    ],
    [
        "Eu não (.*)",
        ["Você realmente não %1?", "Por que você não %1?", "Você quer %1?"],
    ],
    [
        "Eu sinto (.*)",
        [
            "Bom, me conte mais sobre esses sentimentos.",
            "Você costuma se sentir %1?",
            "Quando você costuma se sentir %1?",
            "Quando você se sente %1, o que você faz?",
        ],
    ],
[
        "Eu tenho (.*)",
        [
            "Por que você me diz que tem %1?",
            "Você realmente %1?",
            "Agora que você tem %1, o que você fará a seguir?",
        ],
    ],
    [
        "Eu poderia (.*)",
        [
            "Você poderia explicar por que você faria %1?",
            "Por que você %1?",
            "Quem mais sabe que você faria %1?",
        ],
    ],
    [
        "Existe (.*)",
        [
            "Você acha que existe %1?",
            "É provável que haja %1.",
            "Você gostaria que houvesse %1?",
        ],
    ],
    [
        "Meu (.*)",
        [
            "Entendo, seu %1.",
            "Por que você diz que seu %1?",
            "Quando seu %1, como você se sente?",
        ],
    ],
    [
        "Você (.*)",
        [
            "Nós deveríamos estar discutindo você, não eu.",
            "Por que você diz isso sobre mim?",
            "Por que você se importa se eu %1?",
        ],
    ],
    ["Por que (.*)", ["Por que você não me diz o motivo do %1?", "Por que você acha que %1?"]],
    [
        "Eu quero (.*)",
        [
            "O que significaria para você se você obtivesse %1?",
            "Por que você quer %1?",
            "O que você faria se tivesse %1?",
            "Se você obtivesse %1, então o que você faria?",
        ],
    ],
    [
        "(.*) mãe(.*)",
        [
            "Conte-me mais sobre sua mãe.",
            "Como era sua relação com sua mãe?",
            "Como você se sente sobre sua mãe?",
            "Como isso se relaciona com seus sentimentos hoje?",
            "Boas relações familiares são importantes.",
        ],
    ],
    [
        "(.*) pai(.*)",
        [
            "Conte-me mais sobre seu pai.",
            "Como seu pai fez você se sentir?",
            "Como você se sente em relação ao seu pai?",
            "Seu relacionamento com seu pai se relaciona com seus sentimentos hoje?",
            "Você tem dificuldade em demonstrar afeto com sua família?",
        ],
    ],
    [
    "(.*) filho(.*)",
        [
            "Você tinha amigos íntimos quando criança?",
            "Qual é a tua memória favorita de infância?",
            "Você se lembra de algum sonho ou pesadelo da infância?",
            "As outras crianças às vezes te provocavam?",
            "Como você acha que suas experiências de infância se relacionam com seus sentimentos hoje?",
        ],
    ],
    [
        "(.*)\\?",
        [
            "Porque perguntas isso?",
            "Por favor, considere se você pode responder sua própria pergunta.",
            "Talvez a resposta esteja dentro de você?",
            "Por que você não me diz?",
        ],
    ],
    [
        "Sair",
        [
            "Obrigado por falar comigo.",
            "Adeus.",
            "Obrigado, serão $ 150. Tenha um bom dia!",
        ],
    ],
    [
        "(.*)",
        [
            "Por favor me conte mais.",
            "Vamos mudar um pouco o foco... Conte-me sobre sua família.",
            "Você pode elaborar sobre isso?",
            "Por que você diz isso %1?",
            "Eu vejo.",
            "Muito interessante.",
            "%1.",
            "Entendo. E o que isso te diz?",
            "Como isso faz você se sentir?",
            "Como você se sente quando diz isso?",
        ],
    ],
]
  
def translate(str_):
    """Replace any words found in the reflections dict with their
    corresponding value, e.g. 'I am' --> 'you are'."""
    reflections = {
        "sou": "são",
        "era": "eram",
        "Eu": "você",
        "eu": "você faria",
        "eu tenho": "você tem",
        "eu vou": "você vai",
        "meu": "seu",
        "são": "estou",
        "você": "Eu tenho",
        "você": "Eu vou",
        "seu": "meu",
        "seu": "meu",
        "você": "me",
        "eu": "você",
    }
    return " ".join(reflections.get(word, word) for word in str_.lower().split())


def respond(str_):
    """Take a string input, match it against one of the PATTERNS,
    and complete then return one of the corresponding responses.
    """
    # find a match among keys
    for pattern, responses in PATTERNS:
        match = re.match(pattern, str_)
        if match:
            # found a match ... stuff with corresponding value
            # chosen randomly from among the available options
            resp = random.choice(responses)
            # we've got a response... stuff in reflected text where indicated
            pos = resp.find("%")
            while pos > -1:
                num = int(resp[pos + 1 : pos + 2])
                resp = resp[:pos] + translate(match.group(num)) + resp[pos + 2 :]
                pos = resp.find("%")
            # fix munged punctuation at the end
            if resp[-2:] == "?.":
                resp = resp[:-2] + "."
            if resp[-2:] == "??":
                resp = resp[:-2] + "?"
            return resp


if __name__ == "__main__":
    print("Therapist\n---------")
    print("Talk to the program by typing in plain English, using normal upper-")
    print('and lower-case letters and punctuation.  Enter "quit" when done.')
    print("=" * 72)
    print("Olá. Como você está se sentindo hoje?")
    s = ""
    while s != "quit":
        try:
            s = input("> ")
        except EOFError:
            s = "quit"
        time.sleep(random.uniform(0.1, 0.4))
        print(respond(s.rstrip("!.")))


Therapist
---------
Talk to the program by typing in plain English, using normal upper-
and lower-case letters and punctuation.  Enter "quit" when done.
Olá. Como você está se sentindo hoje?
não muito bem.
OK, mas você pode elaborar um pouco?
Você parece ter certeza.
Como isso faz você se sentir?
Vamos mudar um pouco o foco... Conte-me sobre sua família.
Quando seu pai faleceu ano passado teve câncer e covid, como você se sente?
Como você se sente quando diz isso?
Como você se sente quando diz isso?
Eu vejo.
Você parece ter certeza.
Você parece ter certeza.
tenho.
Boas relações familiares são importantes.
Você parece ter certeza.
Por que você diz isso e o trabalho não vai bem?
Você veio até mim porque você é limitado e tenho problemas de relacionamento, não está dando muito certo?
Você parece ter certeza.
Como você se sente quando diz isso?
Como você se sente quando diz isso?
Eu vejo.
Você parece ter certeza.
Por favor, considere se você pode responder sua própria pergunta.
Por que voc